In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, train_test_split, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

class Classification:
    def __init__(self, path='C.csv', clf_opt='knn', no_of_selected_features=None):
        self.path = path
        self.clf_opt = clf_opt
        self.no_of_selected_features = int(no_of_selected_features) if no_of_selected_features is not None else None

    def classification_pipeline(self):
        if self.clf_opt == 'knn':
            print('\n\t### Training K-Nearest Neighbors (KNN) Classifier ### \n')
            clf = KNeighborsClassifier()
            clf_parameters = {
                'clf__n_neighbors': [3,4,5,6,7,8,9,10,11,12,13,14,15],
                'clf__weights': ['uniform', 'distance'],
                'clf__p': [1, 2],  # 1 for Manhattan distance, 2 for Euclidean distance
            }
        else:
            print('Select a valid classifier \n')
            sys.exit(0)
        return clf, clf_parameters

    def get_class_statistics(self, labels):
        class_statistics = Counter(labels)
        print('\n Class \t\t Number of Instances \n')
        for item in list(class_statistics.keys()):
            print('\t' + str(item) + '\t\t\t' + str(class_statistics[item]))

    def get_data(self, filename):
        reader = pd.read_csv(self.path)
        data = reader.iloc[:, :-1]
        labels = reader['Outcome']
        self.get_class_statistics(labels)
        return data, labels

    def train_classifier(self, X_train, y_train):
        clf, clf_parameters = self.classification_pipeline()
        pipeline = Pipeline([
            ('feature_selection', SelectKBest(chi2, k=self.no_of_selected_features)),
            ('clf', clf),
        ])
        grid = GridSearchCV(pipeline, clf_parameters, scoring='f1_micro', cv=10)
        grid.fit(X_train, y_train)
        clf = grid.best_estimator_
        
        print("\nBest Parameters:")
        print(grid.best_params_)
        return clf

    def evaluate_classifier(self, clf, X_test, y_test):
        y_pred = clf.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        report = classification_report(y_test, y_pred)
        return accuracy, report

    def predict_unseen_data(self, clf, unseen_data):
        return clf.predict(unseen_data)

    def classify(self):
        data, labels = self.get_data('C.csv')
        data = np.asarray(data)

        skf = StratifiedKFold(n_splits=5)
        predicted_class_labels = []
        actual_class_labels = []
        count = 0
        probs = []

        for train_index, test_index in skf.split(data, labels):
            X_train = data[train_index]
            y_train = labels[train_index]
            X_test = data[test_index]
            y_test = labels[test_index]

            count += 1
            print('Training Phase ' + str(count))
            clf = self.train_classifier(X_train, y_train)

            predicted = clf.predict(X_test)
            predicted_probability = clf.predict_proba(X_test)  # Not applicable for KNN
            print(predicted_probability)

            for item in predicted_probability:
                probs.append(float(max(item)))

            for item in y_test:
                actual_class_labels.append(item)
            for item in predicted:
                predicted_class_labels.append(item)

        class_names = list(Counter(labels).keys())
        class_names = [str(x) for x in class_names]

        print('\n ##### Classification Report on Training Data ##### \n')
        print(classification_report(actual_class_labels, predicted_class_labels, target_names=class_names))

        pr = precision_score(actual_class_labels, predicted_class_labels, average='macro')
        print('\n Precision:\t' + str(pr))

        rl = recall_score(actual_class_labels, predicted_class_labels, average='macro')
        print('\n Recall:\t' + str(rl))

        fm = f1_score(actual_class_labels, predicted_class_labels, average='macro')
        print('\n F1-Score:\t' + str(fm))

        ac = accuracy_score(actual_class_labels, predicted_class_labels)
        print('\n Accuracy:\t' + str(ac))

        # Now, let's say you have a new data file called 'unseen_data.csv'
        unseen_data = pd.read_csv('tst4.csv')  # Change the file name accordingly
        unseen_data_features = np.asarray(unseen_data.iloc[:, :])

        # Predict on unseen data
        predictions_unseen_data = self.predict_unseen_data(clf, unseen_data_features)
        print('\n ##### Predictions on Unseen Data ##### \n')
        print(predictions_unseen_data)
        print(len(predictions_unseen_data))


if __name__ == "__main__":
    classifier = Classification(clf_opt='knn', no_of_selected_features=32)
    classifier.classify()



 Class 		 Number of Instances 

	0			1213
	1			1213
Training Phase 1

	### Training K-Nearest Neighbors (KNN) Classifier ### 


Best Parameters:
{'clf__n_neighbors': 4, 'clf__p': 1, 'clf__weights': 'distance'}
[[1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [0.51986927 0.48013073]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0